# Pipeline

This pipeline is intended to simplify to whole process of loading a dataset, creating gaps of different types in it, imputing the missing data and evaluating the imputation method.
You may want to edit the cells preceded by an <span style="color: red">**EDIT:**</span> sign to fit your needs.

## Install dependencies

In [ ]:
!pip install openpyxl > /dev/null 2>&1
!pip install jupyterlab-widgets > /dev/null 2>&1
!pip install jsfileupload > /dev/null 2>&1
!pip install pyxlsb > /dev/null 2>&1
!pip install sklearn > /dev/null 2>&1
!pip install scipy > /dev/null 2>&1
!pip install scikit_learn > /dev/null 2>&1
!pip install missingpy > /dev/null 2>&1
!pip install fancyimpute > /dev/null 2>&1

## Run the imputations

In [1]:
import ipywidgets as widgets
import random

random_state = 0

filtered_imputers = [
    "median imputation",
    "mode imputation",
    "mean imputation",
#     "fillna",
    "linear_interpolation",
#     "missForest_regressor_MICE",
#     "bayesian_ridge_MICE",
    "forward_fill",
    "Hot deck",
    "KNN k=1",
    "KNN k=5",
    "KNN k=10",
    "KNN k=15",
    "KNN k=20",
    "KNN k=100"
]

to_impute = [
    {
        "conf": "knmi",
        "file": "260 De Bilt.csv",
        "target": "Relative atmospheric humidity",
        "sheet_name": None
    },
    {
        "conf": "knmi",
        "file": "260 De Bilt.csv",
        "target": "Global Radiation",
        "sheet_name": None
    },
    {
        "conf": "knmi",
        "file": "260 De Bilt.csv",
        "target": "Temperature",
        "sheet_name": None
    },
    {
        "conf": "factory zero",
        "file": "099.xlsx",
        "target": "flow_temp",
        "sheet_name": "alklimaHeatPump"
    },
    {
        "conf": "factory zero",
        "file": "099.xlsx",
        "target": "op_mode",
        "sheet_name": "alklimaHeatPump"
    },
    {
        "conf": "factory zero",
        "file": "099.xlsx",
        "target": "power",
        "sheet_name": "smartMeter"
    },
    {
        "conf": "factory zero",
        "file": "099.xlsx",
        "target": "co2",
        "sheet_name": "co2sensor"
    }
]

devNullOutput = widgets.Output()

for it in to_impute:
    # Load the data
    %run helpers/load_data.ipynb
    config_select.value = it["conf"]
    file_select.value = it["file"]
    dfloader.add_targets(it["target"], sheet_name=it["sheet_name"])
    df = dfloader.df

    # Create gaps
    random.seed(random_state)
    %run helpers/create_gaps.ipynb

    # Run the imputers
    %run helpers/impute.ipynb
    for k in filtered_imputers:
        try:
            if k == "Hot deck":
                imputer_select.value = "Hot deck" if it["target"] != "op_mode" else "Hot deck (classification)"
                ext = it["file"][-4:]
                hd_donors_selector.value = [opt for opt in hd_donors_selector.options if opt != it["file"] and opt.endswith(ext)]
                hd_confirm_button.click()
            else:
                imputer_select.value = k

            # Evaluate and save results
            with devNullOutput:
                %run helpers/evaluate.ipynb
                %run helpers/saving.ipynb
                save_eval_btn.click()
                save_seperatly_btn.click()
            print(f"Done running {k} on {it['target']}")
        except:
            print(f"Error running {k} on {it['target']}")


Dropdown(description='Config:', options=('factory zero', 'knmi'), value='factory zero')

HTML(value='')

Dropdown(description='Imputer:', options=('median imputation', 'mode imputation', 'mean imputation', 'fillna',…

HTML(value='')

Done running median imputation on Relative atmospheric humidity
Done running mode imputation on Relative atmospheric humidity
Done running mean imputation on Relative atmospheric humidity
Done running linear_interpolation on Relative atmospheric humidity
Done running forward_fill on Relative atmospheric humidity


...

HTML(value='')

IntProgress(value=0, max=38)

HTML(value='')

IntProgress(value=0, max=28)

Done running Hot deck on co2
Done running KNN k=1 on co2
Done running KNN k=5 on co2
Done running KNN k=10 on co2
Done running KNN k=15 on co2
Done running KNN k=20 on co2
Done running KNN k=100 on co2


In [2]:
from pipetorch import flight_passengers
from pipetorch.train import *
from pipetorch.data import PTDataFrame
import time
from datetime import datetime, timedelta
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler
from trainer.DataFrameLoader import *
import sys
import ipywidgets as widgets
import random
from tqdm.notebook import trange
import json

rnn_config = {
  "random_state": 0,
  "batch_size": 5,
  "input_window_size": 12,
  "output_window_size": 1,
  "hidden_size": 95,
  "num_layers": 1,
  "loss": "MSELoss",
  "rnn": "GRU"
}


def factoryzero_date_parser(df: pd.DataFrame) -> pd.DataFrame:
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], unit='s').round('min')
    df = df.set_index("Timestamp")
    return df


def knmi_date_parser(df: pd.DataFrame) -> pd.DataFrame:
    df = df.set_index(["Date(YYYYMMDD)", "Hour"])
    def transform_index(index):
        date, hour = index
        return pd.to_datetime(str(date) + str(hour - 1), format='%Y%m%d%H')
    df = df.set_index(df.index.map(transform_index))
    return df


to_impute = [
    {
        "conf": "knmi",
        "file": "260 De Bilt.csv",
        "date_parser": knmi_date_parser,
        "target": ("0", "Temperature"),
        "features": [
            ("0", "Temperature"),
            ("0", "Global Radiation"),
            ("0", "Dew Temperature"),
            ("0", "Relative atmospheric humidity")
        ]
    },
    {
        "conf": "knmi",
        "file": "260 De Bilt.csv",
        "date_parser": knmi_date_parser,
        "target": ("0", "Relative atmospheric humidity"),
        "features": [
            ("0", "Relative atmospheric humidity"),
            ("0", "Sunshine duration"),
            ("0", "Global Radiation"),
            ("0", "Horizontal visibility")
        ]
    },
    {
        "conf": "knmi",
        "file": "260 De Bilt.csv",
        "date_parser": knmi_date_parser,
        "target": ("0", "Global Radiation"),
        "features": [
            ("0", "Global Radiation"),
            ("0", "Relative atmospheric humidity"),
            ("0", "Temperature"),
            ("0", "Sunshine duration")
        ]
    },
    {
        "conf": "factory zero",
        "file": "099.xlsx",
        "date_parser": factoryzero_date_parser,
        "target": ("alklimaHeatPump", "flow_temp"),
        "features": [
            ("alklimaHeatPump", "flow_temp"),
            ("alklimaHeatPump", "return_temp"),
            ("energyHeatpump", "power")
        ]
    },
    {
        "conf": "factory zero",
        "file": "099.xlsx",
        "date_parser": factoryzero_date_parser,
        "target": ("alklimaHeatPump", "op_mode"),
        "features": [
            ("alklimaHeatPump", "op_mode"),
            ("ventilation", "outdoor_temp")
        ]
    },
    {
        "conf": "factory zero",
        "file": "099.xlsx",
        "date_parser": factoryzero_date_parser,
        "target": ("smartMeter", "power"),
        "features": [
            ("smartMeter", "power"),
            ("ventilation", "outdoor_temp"),
            ("ventilation", "room_temp"),
            ("solar", "power")
        ]
    },
    {
        "conf": "factory zero",
        "file": "099.xlsx",
        "date_parser": factoryzero_date_parser,
        "target": ("co2sensor", "co2"),
        "features": [
            ("co2sensor", "co2")
        ]
    }
]


def to_sequence(x, y, input_window, output_window, device=torch.device('cpu')):
    num_samples = x.shape[0] - input_window - output_window
    def generator():
        nonlocal num_samples, x, y, input_window, output_window, device
        for i in range(num_samples):
            start_x = i
            end_x = start_x + input_window
            X = x[start_x:end_x].values[:]

            start_y = i + input_window
            end_y = start_y + output_window
            Y = y[start_y:end_y].values[:]

            X_tensor = torch.from_numpy(np.array([X], dtype=np.float32)).to(device)
            Y_tensor = torch.from_numpy(np.array([Y], dtype=np.float32)).to(device)
            yield X_tensor, Y_tensor
    return num_samples, generator()

df = None
dfs_with_gaps = []
imputed_dfs = []

for field in to_impute:
    print(f"Imputing {field['target'][0]} - {field['target'][1]}")
    # Set random state
    torch.manual_seed(rnn_config["random_state"])
    np.random.seed(rnn_config["random_state"])
    random.seed(rnn_config["random_state"])

    devNullOutput = widgets.Output()

    # Load the data
    %run helpers/load_data.ipynb
    config_select.value = field["conf"]
    file_select.value = field["file"]

    dfloader.add_targets(field['target'][1], sheet_name=field['target'][0])
    for feature in field['features']:
        dfloader.add_features(feature[1], sheet_name=feature[0])

#     time_column_name = "Timestamp"
#     dfloader.add_index_as_feature(time_column_name)

    df = dfloader.df
    # drop nan values
    df.dropna(axis = 0, inplace = True)

#     def roundTime(dt=None, roundTo=100):
#         # Converting numpy.datetime64 to datetime.datetime
#         ts = pd.Timestamp(dt)
#         dt = ts.to_pydatetime()
#         seconds = (dt.replace(tzinfo=None) - dt.min).seconds
#         rounding = (seconds + roundTo / 2) // roundTo * roundTo
#         return dt + timedelta(0, rounding - seconds, -dt.microsecond)

#     df[time_column_name] = df[time_column_name].apply(roundTime)
#     df[time_column_name] = pd.to_numeric(df[time_column_name])
    
    # Define the model
    class RNN(nn.Module):
        def __init__(self, input_size, hidden_size=100, num_layers=1, output_size=1, rnn=nn.LSTM):
            super().__init__()
            self.l1 = rnn(input_size, hidden_size, num_layers, batch_first=True)
            self.l2 = nn.Linear(hidden_size, output_size)

        def forward(self, X):
            h, _ = self.l1(X)
            h = h[:,-1, :]
            y = self.l2(h)
            y = y + X[:,-1,-1:]
            return y

    rnns = {m.__name__: m for m in [nn.LSTM, nn.GRU]}
    model = RNN(
        input_size=len(df.columns),
        hidden_size=rnn_config["hidden_size"],
        num_layers=rnn_config["num_layers"],
        output_size=1,
        rnn=rnns[rnn_config["rnn"]]
    )
    model_file = f'../models/models-with-none/rnn-{field["target"][1].replace(" ", "_")}.pt'
    model.load_state_dict(torch.load(model_file, map_location=torch.device('cpu')))

    # Create gaps
    %run helpers/create_gaps.ipynb

    imputed_dfs = []
    imputer_configs = []
    imputer = {
        'title': 'RNN'
    }
    imputation_date = datetime.now().replace(second=0, microsecond=0)
    random_state = rnn_config["random_state"]

    iw = rnn_config["input_window_size"]
    ow = rnn_config["output_window_size"]

    for i in range(len(dfs_with_gaps)):
        df_with_gaps = dfs_with_gaps[i]
        imputer_configs.append(json.dumps(rnn_config))
        scaler = StandardScaler()
        data = pd.DataFrame(data=scaler.fit_transform(df_with_gaps.values), columns=df_with_gaps.columns, index=df_with_gaps.index)
        x = data[dfloader.features]
        # interpolate nan values in the first `input_window_size` rows, as they cannot be imputed using RNN (no valid input sequence)
        x.iloc[:iw] = x.iloc[:iw].interpolate(method='time', limit_direction="both")
        y = data[dfloader.targets]
        num_samples, sequence = to_sequence(x, y, iw, ow)
        with trange(num_samples) as tr:
            for i in tr:
                X, Y = next(sequence)
                if not np.isnan(Y): # does not needs prediction
                    continue
                y_index = x.index[i + iw]
                x.loc[y_index, dfloader.targets[0]] = model(X).item()
        imputed_df = pd.DataFrame(data=scaler.inverse_transform(x.values), columns=df_with_gaps.columns, index=df_with_gaps.index)
        imputed_dfs.append(imputed_df)

    with devNullOutput:
        %run helpers/evaluate.ipynb
        %run helpers/saving.ipynb
        save_eval_btn.click()
        save_seperatly_btn.click()

Imputing 0 - Temperature


Dropdown(description='Config:', options=('factory zero', 'knmi'), value='factory zero')

HTML(value='')

  0%|          | 0/17531 [00:00<?, ?it/s]

  0%|          | 0/17531 [00:00<?, ?it/s]

...

  0%|          | 0/105074 [00:00<?, ?it/s]

  0%|          | 0/105074 [00:00<?, ?it/s]

  0%|          | 0/105074 [00:00<?, ?it/s]

  0%|          | 0/105074 [00:00<?, ?it/s]

  0%|          | 0/105074 [00:00<?, ?it/s]

In [3]:
!cp -r saved_results saved_results_`date +%s`